# 1. Package preparation

In [3]:
#! pip install missingno
#! pip install geopy
# !pip install geopandas
# !pip install geoplot   # need conda install -c conda-forge cartopy 
# !pip install shapely
import numpy as np
import pandas as pd
import geopy
np.set_printoptions(threshold=np.inf)
from scipy import stats
import seaborn as sns
%matplotlib inline
from matplotlib.pylab import plt
%pylab inline
import missingno as msno 
import warnings # ignore the warnings
# from geopandas import GeoDataFrame
# import geopandas as gpd
# from shapely.geometry import Point, Polygon
# from geopy.geocoders import Nominatim
# import geoplot as gplt
from string import ascii_letters
import seaborn as sns
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [4]:
UShousing2 = pd.read_csv('UShousing2.csv')
UShousing2.shape

UShousing3 = UShousing2.copy()
X = UShousing3.drop('rent_amount',axis=1)
y = UShousing2['rent_amount']


# 2. Feature Engineering
UShousing2 = UShousing2[UShousing2['rent_amount'] > 0]
xreg = pd.get_dummies(data=X[X.columns.difference(["identity"])])

# 3. Linear Regession Model Implement
##### Split Training set & Testing set
X_train, X_test, y_train, y_test = train_test_split(xreg, y, test_size=0.3, random_state= 42)

numerics = ['int64','float64']
train_num = X_train.select_dtypes(include=numerics)
test_num =  X_test[train_num.columns]


scaler = StandardScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(train_num),columns = train_num.columns)
x_test_scaled = pd.DataFrame(scaler.transform(test_num), columns = test_num.columns)


X_train.loc[:,'bathroom_count'] = x_train_scaled.loc[:,'bathroom_count'].values
X_train.loc[:,'bedroom_count'] = x_train_scaled.loc[:,'bedroom_count'].values
X_train.loc[:,'rent_duration'] = x_train_scaled.loc[:,'rent_duration'].values
X_test.loc[:,'bathroom_count'] = x_test_scaled.loc[:,'bathroom_count'].values
X_test.loc[:,'bedroom_count'] = x_test_scaled.loc[:,'bedroom_count'].values
X_test.loc[:,'rent_duration'] = x_test_scaled.loc[:,'rent_duration'].values

# 2. UI

In [5]:
# !pip install mlxtend

In [6]:
'''# Stepwise
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

# Build step forward feature selection
sfs1 = sfs(clf,k_features = 6,forward=True,floating=False, scoring='r2',cv=3)
# Build step forward feature selection
#sfs1 = SequentialFeatureSelector(clf, n_features_to_select=3,scoring='r2',direction = "forward")


# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
'''

'# Stepwise\nfrom mlxtend.feature_selection import SequentialFeatureSelector as sfs\nfrom sklearn.linear_model import LinearRegression\nclf = LinearRegression()\n\n# Build step forward feature selection\nsfs1 = sfs(clf,k_features = 6,forward=True,floating=False, scoring=\'r2\',cv=3)\n# Build step forward feature selection\n#sfs1 = SequentialFeatureSelector(clf, n_features_to_select=3,scoring=\'r2\',direction = "forward")\n\n\n# Perform SFFS\nsfs1 = sfs1.fit(X_train, y_train)\n'

## Try the Pickle here 

In [9]:
# The 6 features selected out by Stepwise regression
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.metrics as metrics
Xx = X_train.loc[:,('bathroom_count',
 'bedroom_count',
 'City_geo_New York',
 'Postcode_geo_10019',
 'Postcode_geo_10107',
 'house_type_id_Apartment')]
# Use CV find best n_estimator
regressor = GradientBoostingRegressor(
    max_depth=10000,
    n_estimators=5,
    learning_rate=0.1
)
regressor.fit(X_train, y_train)
errors = [metrics.mean_squared_error(y_test, y_pred) for y_pred in regressor.staged_predict(X_test)] 
best_n_estimators = np.argmin(errors)

result =GradientBoostingRegressor(
    max_depth=10000,
    n_estimators=best_n_estimators,
    learning_rate=0.1).fit(Xx,y_train)


In [10]:
import pickle
from flask import Flask, request, render_template
# Since we have already provided model.pkl in the submission folder, we commented out this part 
with open('model.pkl', 'wb') as file:
    pickle.dump(result, file)

In [11]:
# Create application
app = Flask(__name__)

In [13]:
model = pickle.load(open('model.pkl', 'rb'))

In [14]:
uix = X_test.loc[:,['bathroom_count',
 'bedroom_count',
 'City_geo_New York',
 'Postcode_geo_10019',
 'Postcode_geo_10107',
 'house_type_id_Apartment']]

In [15]:
[uix.iloc[0,:]]

[bathroom_count             1.371084
 bedroom_count              0.057880
 City_geo_New York          0.000000
 Postcode_geo_10019         0.000000
 Postcode_geo_10107         0.000000
 house_type_id_Apartment    0.000000
 Name: 1153, dtype: float64]

In [16]:
# Try if the model is stored properly
print(model.predict([uix.iloc[0,:]]))

[1496.70196795]


Flask starts here

In [17]:
# Bind home function to URL
@app.route('/')
def home():
    return render_template('Rent Prediction.html')

@app.route('/predict',methods=['POST'])
def predict():
    #For rendering results on HTML GUI
    int_features = [float(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    prediction = model.predict(final_features)
    output = round(prediction[0], 2) 
    return render_template('Rent Prediction.html', prediction_text='Rent estimated for this property is :{}'.format(output))

In [18]:
#skip
# html file 
'''
text = 
<html>
    <div class = 'login'>
        <h1>Predict Rent price</h1>
         <h3>Enter the following value to predict the rent price </h3>
     
        <form action = "{{ url_for("predict")}}"method = "post">
            <input type = "text" name = "bathroomcount" placeholder = "Bathroom Count" required = "required" />
            <input type = "text" name = "bedroomcount" placeholder = "Bedroom Count" required = "required" />
            <input type = "text" name = "cityNY" placeholder = "citisNY" required = "required" />
            <input type = "text" name = "Post10019" placeholder = "postis10019" required = "required" />
            <input type = "text" name = "Post10107" placeholder = "Post10107" required = "required" />
            <input type = "text" name = "Housetype" placeholder = "House Type" required = "required" />

            <button type = "submit" class = "btn">predict</button>
        </form>
 
      <br>
      <br>
      {{ prediction_text }}
  
    </div>
<html>
'''

'''
# save the html file
file = open("Rent Prediction.html","w")
file.write(text)
file.close()
'''

'\n# save the html file\nfile = open("Rent Prediction.html","w")\nfile.write(text)\nfile.close()\n'

In [19]:
# run this cell for the first time
#!pip install flask==0.12.2

In [ ]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader = False,port=4996,host="0.0.0.0")

 * Running on http://0.0.0.0:4996/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2021 20:29:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2021 20:29:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2021 20:29:09] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# If the above code doesn't work. Trouble shooting by following cells

In [ ]:
#! conda install nodejs --yes

In [ ]:
#! jupyter nbextension enable --py --sys-prefix widgetsnbextension

## Tryout ends here 